# 作業 : (Kaggle)鐵達尼生存預測
***
https://www.kaggle.com/c/titanic

# 作業1
* 試著使用鐵達尼號的例子，創立兩種以上的群聚編碼特徵( mean、median、mode、max、min、count 均可 )

In [1]:
# 做完特徵工程前的所有準備 (與前範例相同)
import pandas as pd
import numpy as np
import copy
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

data_path = '../part02/'
df = pd.read_csv(data_path + 'titanic_train.csv')

train_Y = df['Survived']
df = df.drop(['PassengerId', 'Survived'] , axis=1)
df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [2]:
# 取一個類別型欄位, 與一個數值型欄位, 做群聚編碼
"""
Your Code Here
"""
# I choice Sex, and SibSp 

df.Sex.isnull().sum()  # there is no any null at Sex column
temp_mean = df.groupby(['Sex'])['SibSp'].mean().reset_index()
temp_mode = df.groupby(['Sex'])['SibSp'].apply(lambda x:x.mode()[0]).reset_index()
temp_median = df.groupby(['Sex'])['SibSp'].median().reset_index()
temp_max = df.groupby(['Sex'])['SibSp'].max().reset_index()
temp = pd.merge(temp_mean, temp_mode, how='left', on='Sex')
temp = pd.merge(temp, temp_median, how='left', on = 'Sex')
temp = pd.merge(temp, temp_max, how='left', on ='Sex')
temp.columns = ['Sex', 'Sex_SibSp_Mean', 'Sex_SibSp_Mode', 'Sex_SibSp_Median', 'Sex_SibSp_Max']
print(temp)
df= pd.merge(df, temp, how='left', on= 'Sex')

      Sex  Sex_SibSp_Mean  Sex_SibSp_Mode  Sex_SibSp_Median  Sex_SibSp_Max
0  female        0.694268               0                 0              8
1    male        0.429809               0                 0              8


In [3]:
#只取 int64, float64 兩種數值型欄位, 存於 num_features 中
num_features = []
for dtype, feature in zip(df.dtypes, df.columns):
    if dtype == 'float64' or dtype == 'int64':
        num_features.append(feature)
print(f'{len(num_features)} Numeric Features : {num_features}\n')

# 削減文字型欄位, 只剩數值型欄位
df = df[num_features]
df = df.fillna(-1)
MMEncoder = MinMaxScaler()
df.head()

9 Numeric Features : ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'Sex_SibSp_Mean', 'Sex_SibSp_Mode', 'Sex_SibSp_Median', 'Sex_SibSp_Max']



,Pclass,Age,SibSp,Parch,Fare,Sex_SibSp_Mean,Sex_SibSp_Mode,Sex_SibSp_Median,Sex_SibSp_Max
0,3,22.0,1,0,7.2500,0.429809,0,0,8
1,1,38.0,1,0,71.2833,0.694268,0,0,8
2,3,26.0,0,0,7.9250,0.694268,0,0,8
3,1,35.0,1,0,53.1000,0.694268,0,0,8
4,3,35.0,0,0,8.0500,0.429809,0,0,8


# 作業2
#  we choice Sex and SibSp
    * 將上述的新特徵，合併原有的欄位做生存率預估，結果是否有改善?
    the initial features with LogisticRegression score is 0.7038635542329971
    the new features with LogisticRegression score is 0.7822774202448624
    the new features with GBClasifier score is 0.8238257003099878

In [4]:
# 原始特徵 + 邏輯斯迴歸
"""
Your Code Here
"""
import warnings
warnings.filterwarnings('ignore')
df_minus = df.drop(['Sex_SibSp_Mean', 'Sex_SibSp_Mode', 'Sex_SibSp_Median', 'Sex_SibSp_Max'], axis=1)

# 原始特徵 + 線性迴歸
train_X = MMEncoder.fit_transform(df_minus)
estimator = LogisticRegression()
print(f' initial features with LogisticRegression score is {cross_val_score(estimator, train_X, train_Y, cv=5).mean()}')

# 新特徵 + 邏輯斯迴歸, and New Features + GBClassifier
"""
Your Code Here
"""
train_X = MMEncoder.fit_transform(df)
print(f' the new features with LogisticRegression score is {cross_val_score(estimator, train_X, train_Y, cv=5).mean()}')

from sklearn.ensemble import GradientBoostingClassifier

estimator = GradientBoostingClassifier()
print(f' the new features with GBClasifier score is {cross_val_score(estimator, train_X, train_Y, cv=5).mean()}')

 initial features with LogisticRegression score is 0.7038635542329971
 the new features with LogisticRegression score is 0.7822774202448624
 the new features with GBClasifier score is 0.8238257003099878


# Try again, and we choice Ticket and age

 initial features with LogisticRegression score is 0.6982266036406296
 the new features with LogisticRegression score is 0.7026830395345465
 the new features with GBClasifier score is 0.7083010879899793

In [5]:
data_path = '../part02/'
df = pd.read_csv(data_path + 'titanic_train.csv')

train_Y = df['Survived']
df = df.drop(['PassengerId', 'Survived'] , axis=1)

In [6]:
# print(len(df.Ticket.unique()))
df.Ticket.isnull().sum()  # great, no null, we don't have to consider what value should be filled in to null column
df.Age=df.Age.fillna(df.Age.mean())
temp_mean = df.groupby(['Ticket'])['Age'].mean().reset_index()
temp_mode = df.groupby(['Ticket'])['Age'].apply(lambda x: x.mode()[0]).reset_index()
temp_median = df.groupby(['Ticket'])['Age'].median().reset_index()
temp_max = df.groupby(['Ticket'])['Age'].max().reset_index()

temp = pd.merge(temp_mean, temp_median, on='Ticket', how='left')
temp = pd.merge(temp, temp_mode, on='Ticket', how='left')
temp = pd.merge(temp, temp_max, on='Ticket', how='left')
temp.columns=['Ticket', 'Ticket_Age_Mean', 'Ticket_Age_Median', 'Ticket_Age_Mode', 'Ticket_Age_Max']
df= pd.merge(df, temp, how='left', on= 'Ticket')

In [7]:
#只取 int64, float64 兩種數值型欄位, 存於 num_features 中
num_features = []
for dtype, feature in zip(df.dtypes, df.columns):
    if dtype == 'float64' or dtype == 'int64':
        num_features.append(feature)
print(f'{len(num_features)} Numeric Features : {num_features}\n')

# 削減文字型欄位, 只剩數值型欄位
df = df[num_features]
df = df.fillna(-1)
MMEncoder = MinMaxScaler()
df.head()

9 Numeric Features : ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'Ticket_Age_Mean', 'Ticket_Age_Median', 'Ticket_Age_Mode', 'Ticket_Age_Max']



,Pclass,Age,SibSp,Parch,Fare,Ticket_Age_Mean,Ticket_Age_Median,Ticket_Age_Mode,Ticket_Age_Max
0,3,22.0,1,0,7.2500,22.0,22.0,22.0,22.0
1,1,38.0,1,0,71.2833,38.0,38.0,38.0,38.0
2,3,26.0,0,0,7.9250,26.0,26.0,26.0,26.0
3,1,35.0,1,0,53.1000,36.0,36.0,35.0,37.0
4,3,35.0,0,0,8.0500,35.0,35.0,35.0,35.0


In [8]:
# 原始特徵 + 邏輯斯迴歸
"""
Your Code Here
"""
import warnings
warnings.filterwarnings('ignore')
df_minus = df.drop(['Ticket_Age_Mean', 'Ticket_Age_Median', 'Ticket_Age_Mode', 'Ticket_Age_Max'], axis=1)

# 原始特徵 + 線性迴歸
train_X = MMEncoder.fit_transform(df_minus)
estimator = LogisticRegression()
print(f' initial features with LogisticRegression score is {cross_val_score(estimator, train_X, train_Y, cv=5).mean()}')


# 新特徵 + 邏輯斯迴歸
"""
Your Code Here
"""
train_X = MMEncoder.fit_transform(df)
print(f' the new features with LogisticRegression score is {cross_val_score(estimator, train_X, train_Y, cv=5).mean()}')

from sklearn.ensemble import GradientBoostingClassifier

estimator = GradientBoostingClassifier()
print(f' the new features with GBClasifier score is {cross_val_score(estimator, train_X, train_Y, cv=5).mean()}')

 initial features with LogisticRegression score is 0.6982266036406296
 the new features with LogisticRegression score is 0.7026830395345465
 the new features with GBClasifier score is 0.7094373794900111
